In [ ]:
import tensorflow as tf

In [ ]:
tf.version.VERSION

'2.3.0'

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL

In [ ]:
# install kaggle api
!pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10/kaggle-1.5.10-cp36-none-any.whl
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [ ]:
!cd /content/drive/MyDrive/kaggle

In [ ]:
!kaggle competitions download -c plant-pathology-2020-fgvc7

plant-pathology-2020-fgvc7.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#!unzip \*.zip  && rm *.zip

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv("/content/train.csv")
train_df['img_name'] = train_df['image_id'] + ".jpg"
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab,img_name
0,Train_0,0,0,0,1,Train_0.jpg
1,Train_1,0,1,0,0,Train_1.jpg
2,Train_2,1,0,0,0,Train_2.jpg
3,Train_3,0,0,1,0,Train_3.jpg
4,Train_4,1,0,0,0,Train_4.jpg


In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train=pd.read_csv("/content/train.csv")
test=pd.read_csv("/content/test.csv")
train['image_id']=train['image_id']+'.jpg'
test['image_id']=test['image_id']+'.jpg'
train.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0.jpg,0,0,0,1
1,Train_1.jpg,0,1,0,0
2,Train_2.jpg,1,0,0,0
3,Train_3.jpg,0,0,1,0
4,Train_4.jpg,1,0,0,0


In [ ]:
train_labels = np.float32(train.loc[:, 'healthy':'scab'].values)

train, val = train_test_split(train, test_size = 0.15)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    fill_mode='nearest',
    shear_range=0.1,
    rescale=1/255,
    brightness_range=[0.5, 1.5])

In [ ]:
train_generator=train_datagen.flow_from_dataframe(train,directory='/content/images',
                                                      target_size=(384,384),
                                                      x_col="image_id",
                                                      y_col=['healthy','multiple_diseases','rust','scab'],
                                                      class_mode='raw',
                                                      shuffle=False,
                                                       subset='training',
                                                      batch_size=32)

Found 1547 validated image filenames.


In [ ]:
val_generator=train_datagen.flow_from_dataframe(val,directory='/content/images',
                                                      target_size=(384,384),
                                                      x_col="image_id",
                                                      y_col=['healthy','multiple_diseases','rust','scab'],
                                                      class_mode='raw',
                                                      shuffle=False,
                                                      batch_size=32,
                                                  )

Found 274 validated image filenames.


In [ ]:
test_generator=train_datagen.flow_from_dataframe(test,directory='/content/images',
                                                      target_size=(384,384),
                                                      x_col="image_id",
                                                      y_col=None,
                                                      class_mode=None,
                                                      shuffle=False,
                                                      batch_size=32)

Found 1821 validated image filenames.


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
from keras import optimizers
model_finetuned = ResNet50(include_top=False, weights='imagenet', input_shape=(384,384,3))
x = model_finetuned.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(4, activation="softmax")(x)
model_finetuned = Model(inputs=model_finetuned.input, outputs=predictions)
model_finetuned.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])
model_finetuned.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 384, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 390, 390, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 192, 192, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 192, 192, 64) 256         conv1_conv[0][0]                 
______________________

In [ ]:
from keras.callbacks import ReduceLROnPlateau

In [25]:
STEPS = 384//32
history_1 = model_finetuned.fit_generator(train_generator,                                    
                                  steps_per_epoch=STEPS, 
                                  epochs=25,validation_data=val_generator,validation_steps=100
                                  ,verbose=1,callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=3, min_lr=0.000001)],use_multiprocessing=False,
               shuffle=True)

Epoch 1/25
12/12 [==============================] - 47s 4s/step - loss: 0.3289 - accuracy: 0.8880 - val_loss: 1.7781 - val_accuracy: 0.2920
Epoch 2/25
12/12 [==============================] - 26s 2s/step - loss: 0.3636 - accuracy: 0.8750
Epoch 3/25
12/12 [==============================] - 25s 2s/step - loss: 0.2979 - accuracy: 0.9229
Epoch 4/25
12/12 [==============================] - 24s 2s/step - loss: 0.4471 - accuracy: 0.8540
Epoch 5/25
12/12 [==============================] - 26s 2s/step - loss: 0.3328 - accuracy: 0.9036
Epoch 6/25
12/12 [==============================] - 26s 2s/step - loss: 0.2956 - accuracy: 0.9115
Epoch 7/25
12/12 [==============================] - 26s 2s/step - loss: 0.2683 - accuracy: 0.9115
Epoch 8/25
12/12 [==============================] - 26s 2s/step - loss: 0.2314 - accuracy: 0.9427
Epoch 9/25
12/12 [==============================] - 26s 2s/step - loss: 0.2019 - accuracy: 0.9375
Epoch 10/25
12/12 [==============================] - 26s 2s/step - loss: 0.2

In [26]:
history_1.model.save("PlantPathology")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: PlantPathology/assets


In [27]:
!ls

drive	PlantPathology			sample_data	       test.csv
images	plant-pathology-2020-fgvc7.zip	sample_submission.csv  train.csv


In [28]:
!zip -r /content/saved_model.zip /content/PlantPathology

  adding: content/PlantPathology/ (stored 0%)
  adding: content/PlantPathology/variables/ (stored 0%)
  adding: content/PlantPathology/variables/variables.index (deflated 81%)
  adding: content/PlantPathology/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/PlantPathology/assets/ (stored 0%)
  adding: content/PlantPathology/saved_model.pb (deflated 93%)


In [29]:
fig = go.Figure(data=[
    go.Line(name='train_acc', x=history_1.epoch, y=history_1.history['accuracy']),
    go.Line(name='Val_acc', x=history_1.epoch, y=history_1.history['val_accuracy'])])

fig.update_layout(
    title="Accuracy",
    xaxis_title="epoch",
    yaxis_title="accuracy",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    ))
fig

/usr/local/lib/python3.6/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [30]:
history_1.history

{'accuracy': [0.8880208134651184,
  0.875,
  0.9228650331497192,
  0.8539944887161255,
  0.9036458134651184,
  0.9114583134651184,
  0.9114583134651184,
  0.9427083134651184,
  0.9375,
  0.921875,
  0.9114583134651184,
  0.9311294555664062,
  0.8854166865348816,
  0.9270833134651184,
  0.9201101660728455,
  0.9244791865348816,
  0.9166666865348816,
  0.9311294555664062,
  0.8880208134651184,
  0.9401041865348816,
  0.9348958134651184,
  0.9036458134651184,
  0.93388432264328,
  0.9609375,
  0.9140625],
 'loss': [0.328865647315979,
  0.3635503351688385,
  0.29787197709083557,
  0.4470544755458832,
  0.33279240131378174,
  0.29558101296424866,
  0.26831796765327454,
  0.2314487099647522,
  0.2019031047821045,
  0.2459580898284912,
  0.30034828186035156,
  0.2526295483112335,
  0.3054027855396271,
  0.22147488594055176,
  0.2866334617137909,
  0.266012966632843,
  0.27559682726860046,
  0.2183227390050888,
  0.3509487807750702,
  0.21242259442806244,
  0.23544608056545258,
  0.26072821021